# 이데일리 전처리

In [2]:
import pandas as pd
import re

df = pd.read_csv("naver(이데일리).csv", parse_dates=['time'])
texts = df.loc[:, 'body']

In [14]:
pre_texts = []

for idx, text in enumerate(texts):
    cleaned_text = re.sub('이데일리', '', str(text))
    cleaned_text = re.sub('\[.*] +', '', str(text))
    cleaned_text = re.sub('Copyright.*', '', str(cleaned_text))
    cleaned_text = re.sub('[a-zA-z0-9]+@.+', '', cleaned_text)
    cleaned_text = re.sub("['\n']", '', str(cleaned_text))
    cleaned_text = re.sub("\\\\n]", '', str(cleaned_text))
    cleaned_text = re.sub("[☎=「~」─┬│┴┼↑※:<>◇\-‘’`“”·▶▲,◆!■\+\'\"]", '', str(cleaned_text))
    cleaned_text = re.sub("(\\\\n)", '', cleaned_text)
    cleaned_text = re.sub("(\\\\t)", '', cleaned_text)
    cleaned_text = re.sub("[0-9](\.)[0-9]", '', cleaned_text) #다시    
    cleaned_text = re.sub('＜.*', "", str(cleaned_text))
    cleaned_text = re.sub('☞.*', '', str(cleaned_text))
    cleaned_text = re.sub("[·ㆍ…→\(\)\[\]]", ' ', cleaned_text)
    cleaned_text = re.sub('(https?.+)?[A-Za-z0-9]{1,}\.[A-Za-z0-9]{1,10}\.?[A-Za-z]{1,}\.?[A-Za-z]{1,}', '', cleaned_text)
    cleaned_text = cleaned_text.strip()
    
    if len(re.findall('[가-힣]', cleaned_text)) < 30 :
        cleaned_text = ''

    pre_texts.append(cleaned_text)

In [15]:
df['body']= pre_texts

In [16]:
idx_del = df[df['body'] == ''].index
df = df.drop(idx_del)

In [17]:
idx_del

Int64Index([     2,      9,     22,     24,     44,     46,     48,     54,
                67,     73,
            ...
            101865, 101880, 101883, 101897, 101919, 101924, 101928, 101948,
            101963, 101971],
           dtype='int64', length=7247)

In [7]:
import pandas as pd
from tqdm import tqdm
from ekonlpy.sentiment import MPCK

mpck = MPCK()
# 기사 문장별 split 후 ngramize

ngram_list=[]
for i in tqdm(range(len(df['body']))):
    try:
        sent_ngram=[]
        sent=df['body'][i].split('.')
        
        for s in sent:
            ngrams = mpck.ngramize(mpck.tokenize(s), keep_overlapping_ngram=False)
            
            if ngrams:
                sent_ngram.append(ngrams)
                
        ngram_list.append(sent_ngram)
        
    except:
        print(i)
        
df['ngram']=ngram_list

100%|██████████████████████████████████████████████████████████████████████████| 102005/102005 [57:00<00:00, 29.82it/s]


In [18]:
df.to_csv('이데일리_전처리.csv', encoding='utf-8')